In [1]:
# Initializations
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

import tensorflow as tf
from keras import metrics
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, Concatenate, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, KFold
tf.get_logger().setLevel('ERROR')
import warnings
warnings.filterwarnings('ignore')
import time
import gc

## Tranformation to new scale

In [65]:
D = 0.001
mu = 1e-3
rho = 1000

Re = 150
phi = 4
u_c = (Re * mu) / (rho * D)

os.chdir("/home/zihao/Documents/PhD/Post-Processing/3D/Bidisperse")
dataset = pd.read_csv(f'Arman_PINN/dataset/Re{Re}_phi0{phi}',
                        header=None, delim_whitespace=True).values
for i in range(dataset.shape[0]): 
    dataset[i,-9] *= u_c
    dataset[i,-8] *= u_c
    dataset[i,-7] *= u_c
    V = np.linalg.norm(dataset[i,-9:-6])
    F = 0.5 * rho * (u_c**2) * ((np.pi * D**2) / 4) / (3 * np.pi * mu * D * V)
    T = 0.5 * rho * (u_c**2) * ((np.pi * D**2) / 4) * D / (mu * D**2 * V)
    dataset[i,-1] *= F
    dataset[i,-2] *= F
    dataset[i,-3] *= F
    dataset[i,-4] *= T
    dataset[i,-5] *= T
    dataset[i,-6] *= T
    dataset[i,-7] /= u_c
    dataset[i,-8] /= u_c
    dataset[i,-9] /= u_c
pd.DataFrame(dataset).to_csv(f'Arman_PINN/dataset/new_Re{Re}_phi0{phi}', header=None, index=None, sep=' ')

# Let's start the real game

## $F_x$

In [71]:
%reset -f array
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# free up memory from previous plots_________________
plt.close('all')
plt.clf()

coord = r'F_x' 

# load dataset _______________________________________
Re = 0.2
phi = 2

num_p_incl = 35
N_neurons = 30
epochs = 500
lr = 1.e-2
verbose = 0
batch_size = 128

# coord, title = 'F_x', r'$ Re = ' + f'{Re},' + r' \phi = ' + f'{phi} $'

os.chdir("/home/zihao/Documents/PhD/Post-Processing/3D/Bidisperse")
dataset = pd.read_csv(f'Arman_PINN/raw/new_Re02_phi0{phi}',
                        header=None, delim_whitespace=True).values

col_n = -3

X_in = dataset[:, :-6]
Y_in = dataset[:, [-3]]
Y_in -= Y_in.mean()

HUBER_ALPHA = Y_in.std()
print(HUBER_ALPHA)
    
kf = KFold(n_splits=5, shuffle=True)
train_scores = []
test_scores = []

X_test_plot = np.zeros((0, X_in.shape[1]))
Y_test_plot = np.zeros((0, 1))
Y_pred_test_plot = np.zeros((0, 1))

# from IPython.display import display, Math
# display(Math('$ \Delta ' + coord + ' $'))
# display(Math(title))

print('Input shape:', X_in.shape)
print(f'# of neighbors in the dataset = {int((X_in.shape[1] - 3) / 3)}')
print(f'# of neighbors used for training = {num_p_incl}')
print('________________________________________')

q = 0
Total_time_train = 0
Total_time_test  = 0
# Train-Test split for K-fold cross-validation _______________________________________
for train_index, test_index in kf.split(X_in):
    
    q += 1
    print(f'+++++++++++++++++++++++++Fold {q:1d}+++++++++++++++++++++++++')

    X_train_list = []
    X_test_list = []
    p_input_list = []
    p_out_list = []
    
    X_train, X_test = X_in[train_index], X_in[test_index]
    Y_train, Y_test = Y_in[train_index], Y_in[test_index]
            
    V_in_train = X_train[:, col_n:]
    V_in_test  = X_test[:, col_n:]
    
    # X_train_tmp = X_train
    # for p in range(0, 30):
    #     X_train_tmp[:,3*p+1] = -X_train_tmp[:,3*p+1]
    # X_train = np.vstack((X_train,X_train_tmp))
    # X_train_tmp = X_train
    # for p in range(0, 30):
    #     X_train_tmp[:,3*p+2] = -X_train_tmp[:,3*p+2]
    # X_train = np.vstack((X_train,X_train_tmp))
    # Y_train = np.vstack((Y_train,Y_train))
    # Y_train = np.vstack((Y_train,Y_train))
    # V_in_train_tmp = V_in_train
    # V_in_train_tmp[:,-2] = -V_in_train_tmp[:,-2]
    # V_in_train = np.vstack((V_in_train,V_in_train_tmp))
    # V_in_train_tmp = V_in_train
    # V_in_train_tmp[:,-1] = -V_in_train_tmp[:,-1]
    # V_in_train = np.vstack((V_in_train,V_in_train_tmp))
    
    # Training =======================================
    glorot = tf.keras.initializers.GlorotNormal()
        
    # Custom neural network _______________________________________
    p_shared_layer1 = Dense(N_neurons,
                            activation='elu',
                            kernel_initializer=glorot,
                            name='p_shared1')
    
    p_shared_layer2 = Dense(N_neurons,
                            activation='elu',
                            kernel_initializer=glorot,
                            name='p_shared2')
    
    p_shared_layer3 = Dense(N_neurons,
                            activation='elu',
                            kernel_initializer=glorot,
                            name='p_shared3')
    
    # p_shared_layer4 = Dense(N_neurons,
    #                         activation='elu',
    #                         kernel_initializer=glorot,
    #                         name='p_shared4')
    
#     p_shared_layer5 = Dense(N_neurons,
#                             activation='tanh',
#                             kernel_initializer=glorot,
#                             name='p_shared5')
    
    V_input = Input(shape=V_in_train.shape[1], name='V_in')
    
    V_layer = Dense(N_neurons,
                     activation='elu',
                     kernel_initializer=glorot,
                     name='V_layer1')(V_input)
    
    V_layer = Dense(N_neurons,
                     activation='elu',
                     kernel_initializer=glorot,
                     name='V_layer2')(V_layer)
    
    V_layer = Dense(N_neurons,
                     activation='elu',
                     kernel_initializer=glorot,
                     name='V_layer3')(V_layer)
    
    # V_layer = Dense(N_neurons,
    #                  activation='elu',
    #                  kernel_initializer=glorot,
    #                  name='V_layer4')(V_layer)
    
#     V_layer = Dense(N_neurons,
#                      activation='relu',
#                      kernel_initializer=glorot,
#                      name='V_layer5')(V_layer)
    
    def summation(p_tensor):
        '''
        Each individual neighbor contribution to drag is
        always directed along the x-direction.
        '''
        # Drag _______________________________________
        return tf.reduce_sum(p_tensor, axis=1, keepdims=True)
    #_______________________________________________________________________________

    for p in range(0, num_p_incl):
        X_train_list.append( X_train[:, 3*p:3*p+3])
        p_input_list.append( Input(shape=3, name=f'p_r{p}') )
        # p_out_list.append( tf.keras.layers.Multiply()([tf.keras.layers.Multiply()
        #                   ([p_shared_layer2(p_shared_layer1(p_input_list[p])),V_layer])
        #                                                ,R_input]) )
        # p_out_list.append( tf.keras.layers.Multiply()([p_shared_layer2(p_shared_layer1(p_input_list[p]))
        #                                                ,V_layer]) )
        p_out_list.append( tf.keras.layers.Multiply()([p_shared_layer3(p_shared_layer2(p_shared_layer1(p_input_list[p])))
                                                       ,V_layer]) )
        # p_out_list.append( tf.keras.layers.Multiply()([p_shared_layer4(p_shared_layer3(p_shared_layer2(p_shared_layer1(p_input_list[p]))))
        #                                                ,V_layer]) )

    concat = Concatenate()(
        [Lambda(summation, name=f'summation{j}')(p_out_list[j])\
        for j in range(num_p_incl)])
    
    # concat = tf.keras.layers.Multiply()([concat, V_layer])

    nonneg = tf.keras.constraints.NonNeg()

    # output_layer = Dense(1,
    #                     activation='elu',
    #                     name='out_x',
    #                     )(concat)
    
    output_layer = Dense(1,
                        activation='linear',
                        kernel_constraint=nonneg,
                        name='out_x',
                        use_bias=False
                        )(concat)
    
    # Neural net fitting _______________________________________
    
    def R2_val(y_true, y_pred):
        '''
        This function computes the R^2 for the callback function.
        '''
        SS_res = K.sum(K.square(y_true - y_pred)) 
        SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
        return 1 - SS_res / (SS_tot + K.epsilon())
    
    model = Model(inputs=p_input_list + [V_input], outputs=output_layer)

    # myoptimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    myoptimizer = tf.keras.optimizers.Nadam(learning_rate=lr)
    EarlyStop = EarlyStopping(monitor='val_loss', patience=60, min_delta=5e-5)
    LRDecay = ReduceLROnPlateau(monitor='val_loss', patience=20, factor=0.5, min_delta=1e-4)

    # model.compile(loss='mse', optimizer=myoptimizer, metrics=[metrics.mean_squared_error])
    # model.compile(loss='mean_absolute_error', optimizer=myoptimizer, metrics=[metrics.mean_squared_error])
    he = tf.keras.losses.Huber(delta=HUBER_ALPHA)
    model.compile(loss=he, optimizer=myoptimizer, metrics=R2_val)
    
    history_drag = model.fit(
                            X_train_list + [V_in_train],
                            Y_train,

                            validation_split=0.2,
                            shuffle=True,
                            epochs=epochs,
                            verbose=verbose,
                            batch_size=batch_size,
                            callbacks=[LRDecay,EarlyStop],
                            )
    
    start_time = time.perf_counter()
    Y_pred = model.predict(X_train_list + [V_in_train])
    end_time = time.perf_counter()
    print(f'CPU time: {end_time-start_time:.3f} for {Y_pred.shape[0]:8d} datapoints')
    Total_time_train += end_time-start_time
    
    print(f'Train R^2\t= {r2_score(Y_train, Y_pred):.3f}\n')
    train_scores.append(r2_score(Y_train, Y_pred))
    
    # Testing =======================================
    for p in range(0, num_p_incl):
        X_test_list.append( X_test[:, 3*p:3*p+3] )
        
    start_time = time.perf_counter()
    Y_pred_test = model.predict(X_test_list + [V_in_test])
    end_time = time.perf_counter()
    print(f'CPU time: {end_time-start_time:.3f} for {Y_pred_test.shape[0]:8d} datapoints')
    Total_time_test += end_time-start_time
    
    print(f'Test R^2\t= {r2_score(Y_test, Y_pred_test):.3f}\n')
    test_scores.append(r2_score(Y_test, Y_pred_test))

    # os.chdir("/home/zihao/Documents/PhD/Post-Processing/3D/Bidisperse/Datasets/Paper_3")
    # model.save(f'model_{coord}_Re{Re}_phi{phi}.h5')
    # os.chdir("../../")
    
    print(f'++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')
    
    if q != 5:
        del model
        K.clear_session()   
        gc.collect()
# os.chdir("/home/zihao/Documents/PhD/Post-Processing/3D/Bidisperse/Datasets/Paper_3")
# np.save(f'Re{Re}_phi{phi}_large_true_drag.npy',Y_large_test_plot)
# np.save(f'Re{Re}_phi{phi}_large_pred_drag.npy',Y_large_pred_test_plot)
# np.save(f'Re{Re}_phi{phi}_small_true_drag.npy',Y_small_test_plot)
# np.save(f'Re{Re}_phi{phi}_small_pred_drag.npy',Y_small_pred_test_plot)
# os.chdir("../../")

Average_train_scores = np.mean(train_scores)
Average_test_scores = np.mean(test_scores)

print('========================================\nOverall performance:')
print(f'<Average Train R^2> = {Average_train_scores:.3f}')
print(f'<Average Test R^2> = {Average_test_scores:.3f}')
print(f'<Average Train time> = {(Total_time_train/5):.3f}')
print(f'<Average Test time> = {(Total_time_test/5):.3f}')

print(f'\nTotal number of model parameters: {model.count_params()}\n')
# os.chdir("/home/zihao/Documents/PhD/Post-Processing/3D/Bidisperse/Datasets/Paper_3")
# model.save(f'model_{coord}_Re{Re}_phi{phi}.h5')
# os.chdir("../../")
model.summary()
del model
K.clear_session()   
gc.collect()

0.8136954406728841
Input shape: (3055, 300)
# of neighbors in the dataset = 99
# of neighbors used for training = 35
________________________________________
+++++++++++++++++++++++++Fold 1+++++++++++++++++++++++++
77/77 [==============================] - 1s 7ms/step
CPU time: 52.351 for     2444 datapoints
Train R^2	= 0.834

20/20 [==============================] - 0s 8ms/step
CPU time: 0.504 for      611 datapoints
Test R^2	= 0.815

++++++++++++++++++++++++++++++++++++++++++++++++++++++++

+++++++++++++++++++++++++Fold 2+++++++++++++++++++++++++
77/77 [==============================] - 1s 5ms/step
CPU time: 18.297 for     2444 datapoints
Train R^2	= 0.839

20/20 [==============================] - 0s 6ms/step
CPU time: 0.378 for      611 datapoints
Test R^2	= 0.768

++++++++++++++++++++++++++++++++++++++++++++++++++++++++

+++++++++++++++++++++++++Fold 3+++++++++++++++++++++++++
77/77 [==============================] - 1s 7ms/step
CPU time: 1.078 for     2444 datapoints
Train R^2	= 0.

17820

<Figure size 640x480 with 0 Axes>

## $F_L$

In [28]:
%reset -f array
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# free up memory from previous plots_________________
plt.close('all')
plt.clf()

coord = r'F_L' 

# load dataset _______________________________________
Re = 150
phi = 4

num_p_incl = 35
N_neurons = 30
epochs = 500
lr = 1.e-2
verbose = 0
batch_size = 128

# coord, title = 'F_x', r'$ Re = ' + f'{Re},' + r' \phi = ' + f'{phi} $'

os.chdir("/home/zihao/Documents/PhD/Post-Processing/3D/Bidisperse")
dataset = pd.read_csv(f'Arman_PINN/raw/new_Re{Re}_phi0{phi}',
                        header=None, delim_whitespace=True).values

col_n = -3

X_in = dataset[:, :-6]
if coord == 'F_L':
    Y_in = dataset[:, -2:]
if coord == 'T_L':
    Y_in = dataset[:, -5:-3]
Y_in -= Y_in.mean(axis=0)

HUBER_ALPHA = max(Y_in[:,0].std(),Y_in[:,1].std())
print(HUBER_ALPHA)
    
kf = KFold(n_splits=5, shuffle=True)
train_scores = []
test_scores = []

X_test_plot = np.zeros((0, X_in.shape[1]))
Y_test_plot = np.zeros((0, Y_in.shape[1]))
Y_pred_test_plot = np.zeros((0, Y_in.shape[1]))

# from IPython.display import display, Math
# display(Math('$ \Delta ' + coord + ' $'))
# display(Math(title))

print('Input shape:', X_in.shape)
print(f'# of neighbors in the dataset = {int((X_in.shape[1] - 3) / 3)}')
print(f'# of neighbors used for training = {num_p_incl}')
print('________________________________________')

q = 0
Total_time_train = 0
Total_time_test  = 0
# Train-Test split for K-fold cross-validation _______________________________________
for train_index, test_index in kf.split(X_in):
    
    q += 1
    print(f'+++++++++++++++++++++++++Fold {q:1d}+++++++++++++++++++++++++')

    X_train_list = []
    X_test_list = []
    p_input_list = []
    p_out_list = []
    
    X_train, X_test = X_in[train_index], X_in[test_index]
    Y_train, Y_test = Y_in[train_index], Y_in[test_index]
            
    V_in_train = X_train[:, col_n:]
    V_in_test  = X_test[:, col_n:]
    
    # X_train_tmp = X_train
    # for p in range(0, 30):
    #     X_train_tmp[:,3*p+1] = -X_train_tmp[:,3*p+1]
    # X_train = np.vstack((X_train,X_train_tmp))
    # X_train_tmp = X_train
    # for p in range(0, 30):
    #     X_train_tmp[:,3*p+2] = -X_train_tmp[:,3*p+2]
    # X_train = np.vstack((X_train,X_train_tmp))
    # Y_train = np.vstack((Y_train,Y_train))
    # Y_train = np.vstack((Y_train,Y_train))
    # V_in_train_tmp = V_in_train
    # V_in_train_tmp[:,-2] = -V_in_train_tmp[:,-2]
    # V_in_train = np.vstack((V_in_train,V_in_train_tmp))
    # V_in_train_tmp = V_in_train
    # V_in_train_tmp[:,-1] = -V_in_train_tmp[:,-1]
    # V_in_train = np.vstack((V_in_train,V_in_train_tmp))
    
    # Training =======================================
    glorot = tf.keras.initializers.GlorotNormal()
        
    # Custom neural network _______________________________________
    p_shared_layer1 = Dense(N_neurons,
                            activation='elu',
                            kernel_initializer=glorot,
                            name='p_shared1')
    
    p_shared_layer2 = Dense(N_neurons,
                            activation='elu',
                            kernel_initializer=glorot,
                            name='p_shared2')
    
    p_shared_layer3 = Dense(N_neurons,
                            activation='elu',
                            kernel_initializer=glorot,
                            name='p_shared3')
    
    # p_shared_layer4 = Dense(N_neurons,
    #                         activation='elu',
    #                         kernel_initializer=glorot,
    #                         name='p_shared4')
    
#     p_shared_layer5 = Dense(N_neurons,
#                             activation='tanh',
#                             kernel_initializer=glorot,
#                             name='p_shared5')
    
    V_input = Input(shape=V_in_train.shape[1], name='V_in')
    
    V_layer = Dense(N_neurons,
                     activation='elu',
                     kernel_initializer=glorot,
                     name='V_layer1')(V_input)
    
    V_layer = Dense(N_neurons,
                     activation='elu',
                     kernel_initializer=glorot,
                     name='V_layer2')(V_layer)
    
    V_layer = Dense(N_neurons,
                     activation='elu',
                     kernel_initializer=glorot,
                     name='V_layer3')(V_layer)
    
    # V_layer = Dense(N_neurons,
    #                  activation='elu',
    #                  kernel_initializer=glorot,
    #                  name='V_layer4')(V_layer)
    
#     V_layer = Dense(N_neurons,
#                      activation='relu',
#                      kernel_initializer=glorot,
#                      name='V_layer5')(V_layer)
    
    def project_y(inputs):
        pos_vec = inputs[0]
        p_tensor = inputs[1]

        e_x = tf.constant(np.array([1, 0, 0]), shape=(1, 3), dtype=tf.float32)
        e_y = tf.constant(np.array([0, 1, 0]), shape=(1, 3), dtype=tf.float32)
        e_z = tf.constant(np.array([0, 0, 1]), shape=(1, 3), dtype=tf.float32)

        e_x = tf.broadcast_to(e_x, shape=(tf.shape(p_tensor)[0], 3))
        e_y = tf.broadcast_to(e_y, shape=(tf.shape(p_tensor)[0], 3))
        e_z = tf.broadcast_to(e_z, shape=(tf.shape(p_tensor)[0], 3))

        e_r = pos_vec / tf.linalg.norm(pos_vec, axis=1, keepdims=True)

        # Lift _______________________________________
        if coord == 'F_L':
            e_ctrb = tf.linalg.cross(tf.linalg.cross(e_x, e_r), e_x)

        # Torque _______________________________________
        if coord == 'T_L':
            e_ctrb = tf.linalg.cross(e_x, e_r)

        e_ctrb /= tf.linalg.norm(e_ctrb, axis=1, keepdims=True)
        ctrb = tf.reduce_sum(p_tensor, axis=1, keepdims=True) * e_ctrb

        return tf.reduce_sum( tf.multiply( ctrb, e_y ), axis=1, keepdims=True )
    #_______________________________________________________________________________

    def project_z(inputs):
        pos_vec = inputs[0]
        p_tensor = inputs[1]

        e_x = tf.constant(np.array([1, 0, 0]), shape=(1, 3), dtype=tf.float32)
        e_y = tf.constant(np.array([0, 1, 0]), shape=(1, 3), dtype=tf.float32)
        e_z = tf.constant(np.array([0, 0, 1]), shape=(1, 3), dtype=tf.float32)

        e_x = tf.broadcast_to(e_x, shape=(tf.shape(p_tensor)[0], 3))
        e_y = tf.broadcast_to(e_y, shape=(tf.shape(p_tensor)[0], 3))
        e_z = tf.broadcast_to(e_z, shape=(tf.shape(p_tensor)[0], 3))
        
        e_r = pos_vec / tf.linalg.norm(pos_vec, axis=1, keepdims=True)

        # Lift _______________________________________
        if coord == 'F_L':
            e_ctrb = tf.linalg.cross(tf.linalg.cross(e_x, e_r), e_x)

        # Torque _______________________________________
        if coord == 'T_L':
            e_ctrb = tf.linalg.cross(e_x, e_r)

        e_ctrb /= tf.linalg.norm(e_ctrb, axis=1, keepdims=True)
        ctrb = tf.reduce_sum(p_tensor, axis=1, keepdims=True) * e_ctrb

        return tf.reduce_sum( tf.multiply( ctrb, e_z ), axis=1, keepdims=True )
    #_______________________________________________________________________________

    for p in range(0, num_p_incl):
        X_train_list.append( X_train[:, 3*p:3*p+3])
        p_input_list.append( Input(shape=3, name=f'p_r{p}') )
        # p_out_list.append( tf.keras.layers.Multiply()([tf.keras.layers.Multiply()
        #                   ([p_shared_layer2(p_shared_layer1(p_input_list[p])),V_layer])
        #                                                ,R_input]) )
        # p_out_list.append( tf.keras.layers.Multiply()([p_shared_layer2(p_shared_layer1(p_input_list[p]))
        #                                                ,V_layer]) )
        p_out_list.append( tf.keras.layers.Multiply()([p_shared_layer3(p_shared_layer2(p_shared_layer1(p_input_list[p])))
                                                       ,V_layer]) )
        # p_out_list.append( tf.keras.layers.Multiply()([p_shared_layer4(p_shared_layer3(p_shared_layer2(p_shared_layer1(p_input_list[p]))))
        #                                                ,V_layer]) )

    ctrb_y = Concatenate()(
        [Lambda(project_y, name=f'proj_y{j}')([p_input_list[j], p_out_list[j]])\
        for j in range(num_p_incl)])
    ctrb_z = Concatenate()(
        [Lambda(project_z, name=f'proj_z{j}')([p_input_list[j], p_out_list[j]])\
        for j in range(num_p_incl)])

    nonneg = tf.keras.constraints.NonNeg()

    out_y = Dense(1, activation='linear',
                #   kernel_initializer=tf.keras.initializers.zeros(),
                  kernel_constraint=nonneg,
                name='out_y', use_bias=False)(ctrb_y)
    out_z = Dense(1, activation='linear',
                #   kernel_initializer=tf.keras.initializers.zeros(),
                  kernel_constraint=nonneg,
                name='out_z', use_bias=False)(ctrb_z)
    
    out = Concatenate(name='out_concat_layer')([out_y, out_z])
    
    # Neural net fitting _______________________________________
    
    def R2_val(y_true, y_pred):
        '''
        This function computes the R^2 for the callback function.
        '''
        SS_res = K.sum(K.square(y_true - y_pred)) 
        SS_tot = K.sum(K.square(y_true - K.mean(y_true,axis=0))) 
        return 1 - SS_res / (SS_tot + K.epsilon())
    
    model = Model(inputs=p_input_list + [V_input], outputs=out)

    # myoptimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    myoptimizer = tf.keras.optimizers.Nadam(learning_rate=lr)
    EarlyStop = EarlyStopping(monitor='val_loss', patience=60, min_delta=5e-5)
    LRDecay = ReduceLROnPlateau(monitor='val_loss', patience=20, factor=0.5, min_delta=1e-4)

    # model.compile(loss='mse', optimizer=myoptimizer, metrics=[metrics.mean_squared_error])
    # model.compile(loss='mean_absolute_error', optimizer=myoptimizer, metrics=[metrics.mean_squared_error])
    he = tf.keras.losses.Huber(delta=HUBER_ALPHA)
    model.compile(loss=he, optimizer=myoptimizer, metrics=R2_val)
    
    history_drag = model.fit(
                            X_train_list + [V_in_train],
                            Y_train,

                            validation_split=0.2,
                            shuffle=True,
                            epochs=epochs,
                            verbose=verbose,
                            batch_size=batch_size,
                            callbacks=[LRDecay,EarlyStop],
                            )
    
    start_time = time.perf_counter()
    Y_pred = model.predict(X_train_list + [V_in_train])
    end_time = time.perf_counter()
    print(f'CPU time: {end_time-start_time:.3f} for {Y_pred.shape[0]:8d} datapoints')
    Total_time_train += end_time-start_time
    
    print(f'Train R^2\t= {r2_score(Y_train, Y_pred):.3f}\n')
    train_scores.append(r2_score(Y_train, Y_pred))
    
    # Testing =======================================
    for p in range(0, num_p_incl):
        X_test_list.append( X_test[:, 3*p:3*p+3] )
        
    start_time = time.perf_counter()
    Y_pred_test = model.predict(X_test_list + [V_in_test])
    end_time = time.perf_counter()
    print(f'CPU time: {end_time-start_time:.3f} for {Y_pred_test.shape[0]:8d} datapoints')
    Total_time_test += end_time-start_time
    
    print(f'Test R^2\t= {r2_score(Y_test, Y_pred_test):.3f}\n')
    test_scores.append(r2_score(Y_test, Y_pred_test))

    # os.chdir("/home/zihao/Documents/PhD/Post-Processing/3D/Bidisperse/Datasets/Paper_3")
    # model.save(f'model_{coord}_Re{Re}_phi{phi}.h5')
    # os.chdir("../../")
    
    print(f'++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')
    
    if q != 5:
        del model
        K.clear_session()   
        gc.collect()
# os.chdir("/home/zihao/Documents/PhD/Post-Processing/3D/Bidisperse/Datasets/Paper_3")
# np.save(f'Re{Re}_phi{phi}_large_true_drag.npy',Y_large_test_plot)
# np.save(f'Re{Re}_phi{phi}_large_pred_drag.npy',Y_large_pred_test_plot)
# np.save(f'Re{Re}_phi{phi}_small_true_drag.npy',Y_small_test_plot)
# np.save(f'Re{Re}_phi{phi}_small_pred_drag.npy',Y_small_pred_test_plot)
# os.chdir("../../")

Average_train_scores = np.mean(train_scores)
Average_test_scores = np.mean(test_scores)

print('========================================\nOverall performance:')
print(f'<Average Train R^2> = {Average_train_scores:.3f}')
print(f'<Average Test R^2> = {Average_test_scores:.3f}')
print(f'<Average Train time> = {(Total_time_train/5):.3f}')
print(f'<Average Test time> = {(Total_time_test/5):.3f}')

print(f'\nTotal number of model parameters: {model.count_params()}\n')
# os.chdir("/home/zihao/Documents/PhD/Post-Processing/3D/Bidisperse/Datasets/Paper_3")
# model.save(f'model_{coord}_Re{Re}_phi{phi}.h5')
# os.chdir("../../")
model.summary()
del model
K.clear_session()   
gc.collect()

7.213352694286313
Input shape: (2578, 300)
# of neighbors in the dataset = 99
# of neighbors used for training = 35
________________________________________
+++++++++++++++++++++++++Fold 1+++++++++++++++++++++++++
65/65 [==============================] - 4s 10ms/step
CPU time: 40.800 for     2062 datapoints
Train R^2	= 0.556

17/17 [==============================] - 0s 5ms/step
CPU time: 0.312 for      516 datapoints
Test R^2	= 0.492

++++++++++++++++++++++++++++++++++++++++++++++++++++++++

+++++++++++++++++++++++++Fold 2+++++++++++++++++++++++++
65/65 [==============================] - 3s 4ms/step
CPU time: 3.049 for     2062 datapoints
Train R^2	= 0.594

17/17 [==============================] - 0s 5ms/step
CPU time: 0.282 for      516 datapoints
Test R^2	= 0.498

++++++++++++++++++++++++++++++++++++++++++++++++++++++++

+++++++++++++++++++++++++Fold 3+++++++++++++++++++++++++
65/65 [==============================] - 3s 9ms/step
CPU time: 6.428 for     2062 datapoints
Train R^2	= 0.6

2033

<Figure size 640x480 with 0 Axes>

## $T_L$

In [19]:
%reset -f array
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# free up memory from previous plots_________________
plt.close('all')
plt.clf()

coord = r'T_L' 

# load dataset _______________________________________
Re = 150
phi = 4

num_p_incl = 6
N_neurons = 10
epochs = 500
lr = 8e-3
verbose = 0
batch_size = 128

# coord, title = 'F_x', r'$ Re = ' + f'{Re},' + r' \phi = ' + f'{phi} $'

os.chdir("/home/zihao/Documents/PhD/Post-Processing/3D/Bidisperse")
dataset = pd.read_csv(f'Arman_PINN/raw/new_Re{Re}_phi0{phi}',
                        header=None, delim_whitespace=True).values

col_n = -3

X_in = dataset[:, :-6]
if coord == 'F_L':
    Y_in = dataset[:, -2:]
if coord == 'T_L':
    Y_in = dataset[:, -5:-3]
Y_in -= Y_in.mean(axis=0)

HUBER_ALPHA = max(Y_in[:,0].std(),Y_in[:,1].std())
print(HUBER_ALPHA)
    
kf = KFold(n_splits=5, shuffle=True)
train_scores = []
test_scores = []

X_test_plot = np.zeros((0, X_in.shape[1]))
Y_test_plot = np.zeros((0, Y_in.shape[1]))
Y_pred_test_plot = np.zeros((0, Y_in.shape[1]))

# from IPython.display import display, Math
# display(Math('$ \Delta ' + coord + ' $'))
# display(Math(title))

print('Input shape:', X_in.shape)
print(f'# of neighbors in the dataset = {int((X_in.shape[1] - 3) / 3)}')
print(f'# of neighbors used for training = {num_p_incl}')
print('________________________________________')

q = 0
Total_time_train = 0
Total_time_test  = 0
# Train-Test split for K-fold cross-validation _______________________________________
for train_index, test_index in kf.split(X_in):
    
    q += 1
    print(f'+++++++++++++++++++++++++Fold {q:1d}+++++++++++++++++++++++++')

    X_train_list = []
    X_test_list = []
    p_input_list = []
    p_out_list = []
    
    X_train, X_test = X_in[train_index], X_in[test_index]
    Y_train, Y_test = Y_in[train_index], Y_in[test_index]
            
    V_in_train = X_train[:, col_n:]
    V_in_test  = X_test[:, col_n:]
    
    # X_train_tmp = X_train
    # for p in range(0, 30):
    #     X_train_tmp[:,3*p+1] = -X_train_tmp[:,3*p+1]
    # X_train = np.vstack((X_train,X_train_tmp))
    # X_train_tmp = X_train
    # for p in range(0, 30):
    #     X_train_tmp[:,3*p+2] = -X_train_tmp[:,3*p+2]
    # X_train = np.vstack((X_train,X_train_tmp))
    # Y_train = np.vstack((Y_train,Y_train))
    # Y_train = np.vstack((Y_train,Y_train))
    # V_in_train_tmp = V_in_train
    # V_in_train_tmp[:,-2] = -V_in_train_tmp[:,-2]
    # V_in_train = np.vstack((V_in_train,V_in_train_tmp))
    # V_in_train_tmp = V_in_train
    # V_in_train_tmp[:,-1] = -V_in_train_tmp[:,-1]
    # V_in_train = np.vstack((V_in_train,V_in_train_tmp))
    
    # Training =======================================
    glorot = tf.keras.initializers.GlorotNormal()
        
    # Custom neural network _______________________________________
    p_shared_layer1 = Dense(N_neurons,
                            activation='elu',
                            kernel_initializer=glorot,
                            name='p_shared1')
    
    p_shared_layer2 = Dense(N_neurons,
                            activation='elu',
                            kernel_initializer=glorot,
                            name='p_shared2')
    
    p_shared_layer3 = Dense(N_neurons,
                            activation='elu',
                            kernel_initializer=glorot,
                            name='p_shared3')
    
    # p_shared_layer4 = Dense(N_neurons,
    #                         activation='elu',
    #                         kernel_initializer=glorot,
    #                         name='p_shared4')
    
#     p_shared_layer5 = Dense(N_neurons,
#                             activation='tanh',
#                             kernel_initializer=glorot,
#                             name='p_shared5')
    
    V_input = Input(shape=V_in_train.shape[1], name='V_in')
    
    V_layer = Dense(N_neurons,
                     activation='elu',
                     kernel_initializer=glorot,
                     name='V_layer1')(V_input)
    
    V_layer = Dense(N_neurons,
                     activation='elu',
                     kernel_initializer=glorot,
                     name='V_layer2')(V_layer)
    
    V_layer = Dense(N_neurons,
                     activation='elu',
                     kernel_initializer=glorot,
                     name='V_layer3')(V_layer)
    
    # V_layer = Dense(N_neurons,
    #                  activation='elu',
    #                  kernel_initializer=glorot,
    #                  name='V_layer4')(V_layer)
    
#     V_layer = Dense(N_neurons,
#                      activation='relu',
#                      kernel_initializer=glorot,
#                      name='V_layer5')(V_layer)
    
    def project_y(inputs):
        pos_vec = inputs[0]
        p_tensor = inputs[1]

        e_x = tf.constant(np.array([1, 0, 0]), shape=(1, 3), dtype=tf.float32)
        e_y = tf.constant(np.array([0, 1, 0]), shape=(1, 3), dtype=tf.float32)
        e_z = tf.constant(np.array([0, 0, 1]), shape=(1, 3), dtype=tf.float32)

        e_x = tf.broadcast_to(e_x, shape=(tf.shape(p_tensor)[0], 3))
        e_y = tf.broadcast_to(e_y, shape=(tf.shape(p_tensor)[0], 3))
        e_z = tf.broadcast_to(e_z, shape=(tf.shape(p_tensor)[0], 3))

        e_r = pos_vec / tf.linalg.norm(pos_vec, axis=1, keepdims=True)

        # Lift _______________________________________
        if coord == 'F_L':
            e_ctrb = tf.linalg.cross(tf.linalg.cross(e_x, e_r), e_x)

        # Torque _______________________________________
        if coord == 'T_L':
            e_ctrb = tf.linalg.cross(e_x, e_r)

        e_ctrb /= tf.linalg.norm(e_ctrb, axis=1, keepdims=True)
        ctrb = tf.reduce_sum(p_tensor, axis=1, keepdims=True) * e_ctrb

        return tf.reduce_sum( tf.multiply( ctrb, e_y ), axis=1, keepdims=True )
    #_______________________________________________________________________________

    def project_z(inputs):
        pos_vec = inputs[0]
        p_tensor = inputs[1]

        e_x = tf.constant(np.array([1, 0, 0]), shape=(1, 3), dtype=tf.float32)
        e_y = tf.constant(np.array([0, 1, 0]), shape=(1, 3), dtype=tf.float32)
        e_z = tf.constant(np.array([0, 0, 1]), shape=(1, 3), dtype=tf.float32)

        e_x = tf.broadcast_to(e_x, shape=(tf.shape(p_tensor)[0], 3))
        e_y = tf.broadcast_to(e_y, shape=(tf.shape(p_tensor)[0], 3))
        e_z = tf.broadcast_to(e_z, shape=(tf.shape(p_tensor)[0], 3))
        
        e_r = pos_vec / tf.linalg.norm(pos_vec, axis=1, keepdims=True)

        # Lift _______________________________________
        if coord == 'F_L':
            e_ctrb = tf.linalg.cross(tf.linalg.cross(e_x, e_r), e_x)

        # Torque _______________________________________
        if coord == 'T_L':
            e_ctrb = tf.linalg.cross(e_x, e_r)

        e_ctrb /= tf.linalg.norm(e_ctrb, axis=1, keepdims=True)
        ctrb = tf.reduce_sum(p_tensor, axis=1, keepdims=True) * e_ctrb

        return tf.reduce_sum( tf.multiply( ctrb, e_z ), axis=1, keepdims=True )
    #_______________________________________________________________________________

    for p in range(0, num_p_incl):
        X_train_list.append( X_train[:, 3*p:3*p+3])
        p_input_list.append( Input(shape=3, name=f'p_r{p}') )
        # p_out_list.append( tf.keras.layers.Multiply()([tf.keras.layers.Multiply()
        #                   ([p_shared_layer2(p_shared_layer1(p_input_list[p])),V_layer])
        #                                                ,R_input]) )
        # p_out_list.append( tf.keras.layers.Multiply()([p_shared_layer2(p_shared_layer1(p_input_list[p]))
        #                                                ,V_layer]) )
        p_out_list.append( tf.keras.layers.Multiply()([p_shared_layer3(p_shared_layer2(p_shared_layer1(p_input_list[p])))
                                                       ,V_layer]) )
        # p_out_list.append( tf.keras.layers.Multiply()([p_shared_layer4(p_shared_layer3(p_shared_layer2(p_shared_layer1(p_input_list[p]))))
        #                                                ,V_layer]) )

    ctrb_y = Concatenate()(
        [Lambda(project_y, name=f'proj_y{j}')([p_input_list[j], p_out_list[j]])\
        for j in range(num_p_incl)])
    ctrb_z = Concatenate()(
        [Lambda(project_z, name=f'proj_z{j}')([p_input_list[j], p_out_list[j]])\
        for j in range(num_p_incl)])

    nonneg = tf.keras.constraints.NonNeg()

    out_y = Dense(1, activation='linear',
                #   kernel_initializer=tf.keras.initializers.zeros(),
                  kernel_constraint=nonneg,
                name='out_y', use_bias=False)(ctrb_y)
    out_z = Dense(1, activation='linear',
                #   kernel_initializer=tf.keras.initializers.zeros(),
                  kernel_constraint=nonneg,
                name='out_z', use_bias=False)(ctrb_z)
    
    out = Concatenate(name='out_concat_layer')([out_y, out_z])
    
    # Neural net fitting _______________________________________
    
    def R2_val(y_true, y_pred):
        '''
        This function computes the R^2 for the callback function.
        '''
        SS_res = K.sum(K.square(y_true - y_pred)) 
        SS_tot = K.sum(K.square(y_true - K.mean(y_true,axis=0))) 
        return 1 - SS_res / (SS_tot + K.epsilon())
    
    model = Model(inputs=p_input_list + [V_input], outputs=out)

    myoptimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    # myoptimizer = tf.keras.optimizers.Nadam(learning_rate=lr)
    EarlyStop = EarlyStopping(monitor='val_loss', patience=60, min_delta=1e-5)
    LRDecay = ReduceLROnPlateau(monitor='val_loss', patience=20, factor=0.5, min_delta=5e-5)

    # model.compile(loss='mse', optimizer=myoptimizer, metrics=[metrics.mean_squared_error])
    # model.compile(loss='mean_absolute_error', optimizer=myoptimizer, metrics=[metrics.mean_squared_error])
    he = tf.keras.losses.Huber(delta=HUBER_ALPHA)
    model.compile(loss=he, optimizer=myoptimizer, metrics=R2_val)
    
    history_drag = model.fit(
                            X_train_list + [V_in_train],
                            Y_train,

                            validation_split=0.2,
                            shuffle=True,
                            epochs=epochs,
                            verbose=verbose,
                            batch_size=batch_size,
                            callbacks=[LRDecay,EarlyStop],
                            )
    
    start_time = time.perf_counter()
    Y_pred = model.predict(X_train_list + [V_in_train])
    end_time = time.perf_counter()
    print(f'CPU time: {end_time-start_time:.3f} for {Y_pred.shape[0]:8d} datapoints')
    Total_time_train += end_time-start_time
    
    print(f'Train R^2\t= {r2_score(Y_train, Y_pred):.3f}\n')
    train_scores.append(r2_score(Y_train, Y_pred))
    
    # Testing =======================================
    for p in range(0, num_p_incl):
        X_test_list.append( X_test[:, 3*p:3*p+3] )
        
    start_time = time.perf_counter()
    Y_pred_test = model.predict(X_test_list + [V_in_test])
    end_time = time.perf_counter()
    print(f'CPU time: {end_time-start_time:.3f} for {Y_pred_test.shape[0]:8d} datapoints')
    Total_time_test += end_time-start_time
    
    print(f'Test R^2\t= {r2_score(Y_test, Y_pred_test):.3f}\n')
    test_scores.append(r2_score(Y_test, Y_pred_test))

    # os.chdir("/home/zihao/Documents/PhD/Post-Processing/3D/Bidisperse/Datasets/Paper_3")
    # model.save(f'model_{coord}_Re{Re}_phi{phi}.h5')
    # os.chdir("../../")
    
    print(f'++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')
    
    if q != 5:
        del model
        K.clear_session()   
        gc.collect()
# os.chdir("/home/zihao/Documents/PhD/Post-Processing/3D/Bidisperse/Datasets/Paper_3")
# np.save(f'Re{Re}_phi{phi}_large_true_drag.npy',Y_large_test_plot)
# np.save(f'Re{Re}_phi{phi}_large_pred_drag.npy',Y_large_pred_test_plot)
# np.save(f'Re{Re}_phi{phi}_small_true_drag.npy',Y_small_test_plot)
# np.save(f'Re{Re}_phi{phi}_small_pred_drag.npy',Y_small_pred_test_plot)
# os.chdir("../../")

Average_train_scores = np.mean(train_scores)
Average_test_scores = np.mean(test_scores)

print('========================================\nOverall performance:')
print(f'<Average Train R^2> = {Average_train_scores:.3f}')
print(f'<Average Test R^2> = {Average_test_scores:.3f}')
print(f'<Average Train time> = {(Total_time_train/5):.3f}')
print(f'<Average Test time> = {(Total_time_test/5):.3f}')

print(f'\nTotal number of model parameters: {model.count_params()}\n')
# os.chdir("/home/zihao/Documents/PhD/Post-Processing/3D/Bidisperse/Datasets/Paper_3")
# model.save(f'model_{coord}_Re{Re}_phi{phi}.h5')
# os.chdir("../../")
model.summary()
del model
K.clear_session()   
gc.collect()

4.093980262268184
Input shape: (2578, 300)
# of neighbors in the dataset = 99
# of neighbors used for training = 6
________________________________________
+++++++++++++++++++++++++Fold 1+++++++++++++++++++++++++
65/65 [==============================] - 1s 2ms/step
CPU time: 0.627 for     2062 datapoints
Train R^2	= 0.631

17/17 [==============================] - 0s 2ms/step
CPU time: 0.180 for      516 datapoints
Test R^2	= 0.632

++++++++++++++++++++++++++++++++++++++++++++++++++++++++

+++++++++++++++++++++++++Fold 2+++++++++++++++++++++++++
65/65 [==============================] - 1s 4ms/step
CPU time: 0.721 for     2062 datapoints
Train R^2	= 0.638

17/17 [==============================] - 0s 3ms/step
CPU time: 0.208 for      516 datapoints
Test R^2	= 0.619

++++++++++++++++++++++++++++++++++++++++++++++++++++++++

+++++++++++++++++++++++++Fold 3+++++++++++++++++++++++++
65/65 [==============================] - 0s 2ms/step
CPU time: 0.495 for     2062 datapoints
Train R^2	= 0.636


7633

<Figure size 640x480 with 0 Axes>